In [2]:
import sqlite3 as db
import pandas as pd
import os
import json
import random
import re

---

<center><h2><b>Leer DB</b></h2></center>

De primeras vamos a cargar la base de datos sqlite3 del dataset para leerla y hacernos una idea de a que nos vamos a enfrentar. Además, hemos descomprimido los CSV para cargarlos en un Dataframe y agilizar el proceso de limpieza y pre-procesado.

In [3]:
# Hacer una query SQL
def sql_query(q):
    conn = db.connect('../db/sqlite/eicu_v2_0_1.sqlite3')
    df = pd.read_sql_query(q, conn)
    conn.close()
    
    return df

# Leer todos los CSV que hemos descomprimido
def read_csvs():
   

    datasets = [ 'admissiondrug', 'admissionDx', 'allergy', 'apacheApsVar', 'apachePatientResult', 'apachePredVar', 'carePlanCareProvider', 'carePlanEOL', 'carePlanGeneral',
                 'carePlanGoal', 'carePlanInfectiousDisease', 'customLab', 'diagnosis', 'hospital', 'infusiondrug', 'intakeOutput', 'lab', 'medication', 'microLab', 'note',
                 'nurseAssessment', 'nurseCare', 'nurseCharting', 'pastHistory', 'patient', 'physicalExam', 'respiratoryCare', 'respiratoryCharting', 'treatment', 'vitalAperiodic',
                 'vitalPeriodic']

    dfs = {}

    for ds_name in datasets:
        dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')
    
    return dfs

dfs = read_csvs()


/tmp/ipykernel_27043/1005471908.py:21: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs[ds_name] = pd.read_csv('../db/csv/' + ds_name + '.csv')


---

<center><h2><b>Limpieza del Dataset</b></h2></center>

Una de las primeras decisiones que hemos tomado ha sido dropear todas las ID de las tablas, a excepción de la tabla pacient para hacer los posteriores JOINs, <br>
y la de hostital que a primeras es la única que no presenta patientId y puede que sea necesario en un futuro.

In [4]:
# Dropear IDs de todas las tablas
def drop_keys(dfs):
    for df_key in dfs.keys():
        if df_key not in ('hospital', 'patient'): # No dropear hospitalId o patientUnitStayId
            df = dfs[df_key]
            dfs[df_key] = df.drop(columns=[df.columns.values[0]])

# Para que no se droppeen cada vez que se ejecuta la celda siguiente            
has_dropped_keys = False             

if not has_dropped_keys:
    drop_keys(dfs)
    has_dropped_keys = True

Tras eliminar los ID nos disponemos a dropear las filas duplicadas, para reducir la complejidad del análisis

In [5]:
# Eliminar filas duplicadas
for df_name in dfs:
    dfs[df_name] = dfs[df_name].drop_duplicates()

Tras este primer barrido ahora nos ponemos a explorar cada tabla del conjunto de datos. En base a los datos recogidos y al sentido común trataremos de hacer un buen pre-procesado de los datos.

---

<center><h2><b>admissiondrug</b></h2></center>

**Propósito**: el admissiondrug contiene detalles de los medicamentos que un paciente estaba tomando antes de su ingreso en la UCI. Esta tabla incluye información sobre el medicamento <br>
de admisión para un paciente, como el nombre del medicamento, la dosis, el período de tiempo durante el cual se administró el medicamento, el tipo de usuario y la especialidad del médico que <br>
ingresa los datos, y el tipo de nota donde se ingresó la información.


In [6]:
dfs['admissiondrug'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7417 entries, 0 to 7416
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   patientunitstayid   7417 non-null   int64  
 1   drugoffset          7417 non-null   int64  
 2   drugenteredoffset   7417 non-null   int64  
 3   drugnotetype        7417 non-null   object 
 4   specialtytype       7417 non-null   object 
 5   usertype            7417 non-null   object 
 6   rxincluded          7417 non-null   bool   
 7   writtenineicu       7417 non-null   bool   
 8   drugname            7417 non-null   object 
 9   drugdosage          7417 non-null   float64
 10  drugunit            7417 non-null   object 
 11  drugadmitfrequency  7417 non-null   object 
 12  drughiclseqno       7417 non-null   int64  
dtypes: bool(2), float64(1), int64(4), object(6)
memory usage: 709.8+ KB


In [7]:
dfs['admissiondrug'].head()

,patientunitstayid,drugoffset,drugenteredoffset,drugnotetype,specialtytype,usertype,rxincluded,writtenineicu,drugname,drugdosage,drugunit,drugadmitfrequency,drughiclseqno
0,281479,420,444,Daily Progress,eCM Primary,THC Physician,False,True,NOVOLOG ...,0.0,,,20769
1,281479,24,31,Admission,eCM Primary,THC Nurse,True,True,NOVOLOG ...,0.0,,,20769
2,292154,242,243,Daily Progress,eCM Primary,Other,False,True,ALLOPURINOL ...,0.0,,,1100
3,292154,53,69,Admission,eCM Primary,THC Nurse,False,True,DILTIAZEM 24HR CD ...,0.0,,,182
4,292154,242,243,Daily Progress,eCM Primary,Other,False,True,CALCIUM CARBONATE ...,0.0,,,1163


En este caso la tablas presenta columnas con valores vacios, como drugunit o drugadmitfrequency que van a ser eliminadas. Hay otros valores como drughiclseqno que representan una ID<br>
 del medicamento, esta última será eliminada al ser una variable dependiente al medicamento.

No podemos olvidar que el número de filas es muy elevado ya que tenemos muchas tablas y no es optimo computacionalmente hablando, una solución podría ser intentar conseguir que <br>
solo haya una fila para cada paciente.

Las características más importante que hemos deducido son las relativas al nombre de los medicamentos que tomaban, y su dosis. Sin embargo, la dosis en muchos casos presenta valores <br>
dispares y poco representativos si estudiamos su varianza, otras features con este tipo de circunstancias han sido las que finalmente hemos dropeado ya que nos obstaculizan el pre-proceso de los <br>
medicamentos durante la admisión, nuestros principal objetivo en esta tabla.

In [8]:
dfs['admissiondrug'] = dfs['admissiondrug'].drop(columns=[
    'drugoffset', 'drugenteredoffset', 'drugnotetype', 'specialtytype', 'rxincluded', 'writtenineicu', 'drugunit', 'drugdosage',
    'drugadmitfrequency', 'drughiclseqno', 'usertype'
], axis=1)

In [9]:
dfs['admissiondrug'].head(15)

,patientunitstayid,drugname
0,281479,NOVOLOG ...
1,281479,NOVOLOG ...
2,292154,ALLOPURINOL ...
3,292154,DILTIAZEM 24HR CD ...
4,292154,CALCIUM CARBONATE ...
5,292154,ALLOPURINOL ...
6,292154,ASPIRIN ...
7,292154,CLONIDINE ...
8,292154,COUMADIN ...
9,292154,CHOLECALCIFEROL (VIT D3)/SOY ISOFLAVONE ...


Para poder reducir el numero de filas, hemos considerado que una de las mejores tácticas sería buscar que medicamentos son los más comunes y crear X nuevas columnas que representen si los pacientes <br>
han tomado esos medicamentos.

In [10]:
dfs['admissiondrug']['drugname'].value_counts()[:3]

ASPIRIN                                                                                                                                                                                                                                                            251
LISINOPRIL                                                                                                                                                                                                                                                         206
LASIX                                                                                                                                                                                                                                                              161
Name: drugname, dtype: int64

Hemos normarlizado los datos para que el drugname sea es minusculas.

In [11]:
dfs["admissiondrug"]['drugname']   = dfs["admissiondrug"].apply(lambda row : row['drugname'].lower(), axis=1)

dfs["admissiondrug"]

,patientunitstayid,drugname
0,281479,novolog ...
1,281479,novolog ...
2,292154,allopurinol ...
3,292154,diltiazem 24hr cd ...
4,292154,calcium carbonate ...
...,...,...
7412,3246445,lipitor ...
7413,3246445,mysoline ...
7414,3246445,claritin ...
7415,3246445,flomax ...


In [12]:
# Son los medicamentos más usados en las admisiones
def has_commun_drug_A(row):
    drug = row['drugname'].strip()
       
    if drug == "aspirin":
        return 1
    else:
        return 0

def has_commun_drug_L(row):
    drug = row['drugname'].strip()
    
    if drug == 'lisinopril':
        return 1
    else:
        return 0

def has_commun_drug_Li(row):
    drug = row['drugname'].strip()
    
    if drug in 'lasix':
        return 1
    else:
        return 0

Cada columna va a representar estos tres medicamentos y para cada paciente marcaremos con un 1 si ha tomado o no ha tomado previamente el medicamento.

In [13]:
_dfs = dfs["admissiondrug"]["patientunitstayid"]

# Creamos nuevas columnas
commun_drug_A   = dfs["admissiondrug"].apply(lambda row : has_commun_drug_A(row), axis=1)
commun_drug_L   = dfs["admissiondrug"].apply(lambda row : has_commun_drug_L(row), axis=1)
commun_drug_Li  = dfs["admissiondrug"].apply(lambda row : has_commun_drug_Li(row), axis=1)


_commun_drug_A  = pd.DataFrame(commun_drug_A, columns=['aspirin'])
_commun_drug_L  = pd.DataFrame(commun_drug_L, columns=['lisinopril'])
_commun_drug_Li = pd.DataFrame(commun_drug_Li, columns=['lasix'])


dfs["admissiondrug"] = pd.concat([_dfs, _commun_drug_A,_commun_drug_L,_commun_drug_Li], axis=1)

#Eliminamos columnas duplicadas
dfs['admissiondrug'] = dfs['admissiondrug'].drop_duplicates()
dfs["admissiondrug"] = dfs["admissiondrug"].groupby(['patientunitstayid']).max()

Pora hacer más visual la tabla, cambiamos los 1/0 por True/False.

In [14]:
# Convertir [1,0] to [True, False]
def int_to_boolean (data, column):
    data = dfs["admissiondrug"].copy()                 
    data[column] = data[column].astype(bool)          
    return data

In [15]:
dfs["admissiondrug"] = int_to_boolean (dfs["admissiondrug"], 'aspirin')
dfs["admissiondrug"] = int_to_boolean (dfs["admissiondrug"], 'lisinopril')
dfs["admissiondrug"] = int_to_boolean (dfs["admissiondrug"], 'lasix')
dfs["admissiondrug"] = dfs["admissiondrug"].reset_index()

dfs["admissiondrug"]

,patientunitstayid,aspirin,lisinopril,lasix
0,281479,False,False,False
1,292154,True,False,False
2,310446,False,True,False
3,332296,False,True,False
4,350263,False,False,False
...,...,...,...,...
546,3237226,False,False,False
547,3237558,True,False,False
548,3238529,True,False,True
549,3245076,True,False,False


Finalmente hemos conseguido información única de 551 pacientes, reduciendo el número de filas y eliminando los datos que menos información aportaban.

---

<center><h2><b>admissionDx</b></h2></center>

Propósito: La tabla admissiondx contiene el primer diagnóstico para el ingreso en la UCI según los criterios de puntuación APACHE. <br>
Se introduce en los formularios de notas del paciente. Una vez transcurrido un periodo fijo desde el ingreso, la tabla no puede ser actualizada por el cuidador.

In [16]:
dfs['admissionDx'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7578 entries, 0 to 7577
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   patientunitstayid     7578 non-null   int64 
 1   admitdxenteredoffset  7578 non-null   int64 
 2   admitdxpath           7578 non-null   object
 3   admitdxname           7578 non-null   object
 4   admitdxtext           7578 non-null   object
dtypes: int64(2), object(3)
memory usage: 355.2+ KB


In [17]:
dfs['admissionDx'].head()

,patientunitstayid,admitdxenteredoffset,admitdxpath,admitdxname,admitdxtext
0,2900423,162,admission diagnosis|Non-operative Organ System...,Cardiovascular,Cardiovascular
1,2900423,162,admission diagnosis|Was the patient admitted f...,No,No
2,2900423,162,admission diagnosis|All Diagnosis|Non-operativ...,"Sepsis, pulmonary","Sepsis, pulmonary"
3,2902156,944,admission diagnosis|All Diagnosis|Non-operativ...,"Rhythm disturbance (atrial, supraventricular)","Rhythm disturbance (atrial, supraventricular)"
4,2902156,944,admission diagnosis|Non-operative Organ System...,Cardiovascular,Cardiovascular


Como podemos observar no hay valores nulos en ninguna columna, ya que existen 7578 filas y cada columna tiene 7578 valores non-null. <br>
Por otro lado, admitdxname y admitdxtext tiene la misma información asi que eliminaremos una de las columnas para no cargar al modelo. <br>

Además, la columna admitdxpath es un diagnostico que realiza el medico analizando el estado del patient y termina con una conclusión que es la que <br>
se introduce en la columna admitdxname. <br>

Esto nos ha hecho plantearnos que la información más importante es la de admitdxname. Lo que haremos será estudiar cuales son las más comunes <br>
y hacer una nueva columna con esas admisiones, relacionando el tipo de admisión con el tiempo que ha tardado el medico en diagnosticarlo. <br>

El quedarnos con las más comunes hará que reduzcamos el número de filas ya que existirá una fila por patient al igual que en admissionDrug.



In [18]:
def clean_admitdxname(row):
    admitdxname = row ['admitdxname']
    
    m = re.search('^(\w+)', admitdxname)
    
    return m.group(0)

Para normalizar el admitdxname hemos usado Regular Expressions para quedarnos con la primera palabra. Ej: Sepsis, pulmonary -> Sepsis.

In [19]:
# Drop columns
dfs['admissionDx'] = dfs['admissionDx'].drop(columns=[
    'admitdxtext', # En la gran mayoría de los casos es igual a admitdxname
    'admitdxpath',
], axis=1)

dfs['admissionDx']['admitdxname'] = dfs['admissionDx'].apply(lambda row : clean_admitdxname(row), axis=1)

In [20]:
dfs['admissionDx'].head()

,patientunitstayid,admitdxenteredoffset,admitdxname
0,2900423,162,Cardiovascular
1,2900423,162,No
2,2900423,162,Sepsis
3,2902156,944,Rhythm
4,2902156,944,Cardiovascular


Carece de sentido quedarnos con admitdxname que sean Yes o No ya que buscamos procesar los diagnosticos y estos valores con ambiguos.

In [21]:
# Filtramos y quitamos los Yes y No, no tienen mucho sentido
df = dfs['admissionDx']
dfs['admissionDx'] = df[df['admitdxname'] != 'Yes']
df = dfs['admissionDx']
dfs['admissionDx'] = df[df['admitdxname'] != 'No']

In [22]:
dfs['admissionDx']['admitdxname'].value_counts()[:5]

Cardiovascular      983
Respiratory         363
Neurologic          318
Sepsis              288
Gastrointestinal    264
Name: admitdxname, dtype: int64

Nos quedamos con estas 3 admisiones y establecemos su offset.

In [23]:
def has_common_admission_C(row):
    drug = row ['admitdxname'].strip()
       
    if drug == "Cardiovascular": 
        return row['admitdxenteredoffset']
    else:
        return 0

def has_common_admission_R(row):
    drug = row ['admitdxname'].strip()
    
    if drug == 'Respiratory':
        return row['admitdxenteredoffset']
    else:
        return 0

def has_common_admission_N(row):
    drug = row ['admitdxname'].strip()
    
    if drug in 'Neurologic':
        return row['admitdxenteredoffset']
    else:
        return 0

In [24]:
_dfs = dfs["admissionDx"]["patientunitstayid"]

# Creamos nuevas columnas
common_admission_C   = dfs["admissionDx"].apply(lambda row : has_common_admission_C(row), axis=1)
common_admission_R   = dfs["admissionDx"].apply(lambda row : has_common_admission_R(row), axis=1)
common_admission_N   = dfs["admissionDx"].apply(lambda row : has_common_admission_N(row), axis=1)


_common_admission_C  = pd.DataFrame(common_admission_C, columns=['OffSet_Cardiovascular'])
_common_admission_R  = pd.DataFrame(common_admission_R, columns=['OffSet_Respiratory'])
_common_admission_N  = pd.DataFrame(common_admission_N, columns=['OffSet_Neurologic'])


dfs["admissionDx"] = pd.concat([_dfs, _common_admission_C,_common_admission_R,_common_admission_N], axis=1)

dfs["admissionDx"] = dfs["admissionDx"].groupby(['patientunitstayid']).max()

dfs["admissionDx"] = dfs["admissionDx"].reset_index()

In [25]:
dfs["admissionDx"]

,patientunitstayid,OffSet_Cardiovascular,OffSet_Respiratory,OffSet_Neurologic
0,141765,7,0,0
1,143870,5,0,0
2,144815,0,0,37
3,145427,0,0,0
4,147307,54,0,0
...,...,...,...,...
2216,3351763,0,0,0
2217,3352230,1274,0,0
2218,3352231,5,0,0
2219,3352333,0,0,0


Este es el resultado final del pre-procesado de AdmissionDx.

---

<center><h2><b>allergy</b></h2></center>

**Propósito**: Las tablas de alergias contienen datos sobre las alergias de los pacientes. Los datos se introducen en los formularios de notas de los pacientes.

In [26]:
dfs['allergy'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2475 entries, 0 to 2474
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   patientunitstayid     2475 non-null   int64  
 1   allergyoffset         2475 non-null   int64  
 2   allergyenteredoffset  2475 non-null   int64  
 3   allergynotetype       2475 non-null   object 
 4   specialtytype         2475 non-null   object 
 5   usertype              2475 non-null   object 
 6   rxincluded            2475 non-null   bool   
 7   writtenineicu         2475 non-null   bool   
 8   drugname              1480 non-null   object 
 9   allergytype           2475 non-null   object 
 10  allergyname           2475 non-null   object 
 11  drughiclseqno         1480 non-null   float64
dtypes: bool(2), float64(1), int64(3), object(6)
memory usage: 217.5+ KB


In [27]:
dfs['allergy'].head()

,patientunitstayid,allergyoffset,allergyenteredoffset,allergynotetype,specialtytype,usertype,rxincluded,writtenineicu,drugname,allergytype,allergyname,drughiclseqno
0,243097,2549,2552,Comprehensive Progress,eCM Primary,THC Nurse,True,True,NaN,Non Drug,penicillins,NaN
1,243097,1288,1294,Comprehensive Progress,eCM Primary,THC Nurse,True,True,CODEINE PHOSPHATE,Drug,CODEINE PHOSPHATE,1721.0
2,243097,2549,2552,Comprehensive Progress,eCM Primary,THC Nurse,True,True,CODEINE PHOSPHATE,Drug,CODEINE PHOSPHATE,1721.0
3,243097,21,28,Admission,eCM Primary,THC Nurse,True,True,NaN,Non Drug,penicillins,NaN
4,243097,3988,3989,Comprehensive Progress,eCM Primary,THC Nurse,True,True,CODEINE PHOSPHATE,Drug,CODEINE PHOSPHATE,1721.0


En este caso hay compos nulos en las columnas drugname y drughiclseqno. En este caso la principal información que podemos sacar es si el <br>
posee alguna alergia y si tiene cual sería. Como hay drugnames nullos vamos a dividir la tabla en dos, una para los pacientes con alergía detectadas <br>
y otra sin detectar, quitando Nans y eliminando las columnas que no sean relativas a la identificación de la alergia en los patients.

In [28]:
# Dividir en dos tablas de drogas y no drogas para quitar NaNs y eliminar columnas
dfs['_allergyDrug']    = dfs['allergy'][dfs['allergy']['allergytype'] == 'Drug']\
    .drop(columns=['allergytype', 'allergyname']) # allergyname siempre es igual a drugname
dfs['_allergyNonDrug'] = dfs['allergy'][dfs['allergy']['allergytype'] == 'Non Drug']\
    .drop(columns=['allergytype', 'drugname', 'drughiclseqno'])

# TODO: Some drugs in specific?
patients = {}

for i, row in dfs['_allergyDrug'].iterrows():
    pusID, drugname = row['patientunitstayid'], row['drugname']
    
    if pusID not in patients:
        patients[pusID] = { 'drugs': set(), 'nondrugs': set() }
    
    patients[pusID]['drugs'].add(drugname.lower())

for i, row in dfs['_allergyNonDrug'].iterrows():
    pusID, allergyname = row['patientunitstayid'], row['allergyname']
    
    if pusID not in patients:
        patients[pusID] = { 'drugs': set(), 'nondrugs': set() }
    
    patients[pusID]['nondrugs'].add(allergyname)
    

new_df = pd.DataFrame(columns=['patientunitstayid', 'numero_DrugsAllergic', 'numero_NondrugsAllergic', 'totalAllergic'])
keys = patients.keys()

new_df['patientunitstayid'] = keys
new_df['numero_DrugsAllergic'] = list(map(lambda key: len(patients[key]['drugs']), keys))
new_df['numero_NondrugsAllergic'] = list(map(lambda key: len(patients[key]['nondrugs']), keys))
new_df['totalAllergic'] = new_df['numero_DrugsAllergic'] + new_df['numero_NondrugsAllergic']

dfs['allergy'] = new_df

Este sería el dataset resultante:

In [29]:
dfs['allergy'].head()

,patientunitstayid,numero_DrugsAllergic,numero_NondrugsAllergic,totalAllergic
0,243097,1,1,2
1,244477,3,0,3
2,246997,2,0,2
3,250073,2,2,4
4,260860,2,0,2


---

<center><h2><b>apacheapsvar, apachepatientresult y apachepredvar</b></h2></center>

Las tablas relativas a Apache hemos intentando evitarlas por el miedo de utilizar features que provoquen un futuro overfitting en el entreno, <br>
nuestro objetivo es emular/mejorar las reglas estadísticas que realiza APACHE a través del resto de los datos.

Sin embargo, nos han servido en el resto del pre-procesado ya que han sido nuestras referencias durante la exploración de los datos.

---

<center><h2><b>careplancareprovider</b></h2></center>

**Proposito**: Tiene datos relativos al médico gestor/consultor, incluye la especialidad y la categoría de intervención.

In [30]:
dfs['carePlanCareProvider'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5496 entries, 0 to 5626
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   patientunitstayid       5496 non-null   int64  
 1   careprovidersaveoffset  5496 non-null   int64  
 2   providertype            0 non-null      float64
 3   specialty               4968 non-null   object 
 4   interventioncategory    4110 non-null   object 
 5   managingphysician       5496 non-null   object 
 6   activeupondischarge     5496 non-null   bool   
dtypes: bool(1), float64(1), int64(2), object(3)
memory usage: 305.9+ KB


In [31]:
dfs['carePlanCareProvider'].head()

,patientunitstayid,careprovidersaveoffset,providertype,specialty,interventioncategory,managingphysician,activeupondischarge
0,149713,11,NaN,family practice,I,Managing,True
1,157016,2,NaN,obstetrics/gynecology,I,Managing,True
2,165840,26,NaN,internal medicine,I,Managing,True
3,174826,49,NaN,critical care medicine (CCM),NaN,Managing,True
4,174956,3,NaN,cardiology,Unknown,Managing,True


In [32]:
dfs['carePlanCareProvider']['specialty'].value_counts()[:5]

internal medicine    990
hospitalist          549
cardiology           529
pulmonary/CCM        350
family practice      349
Name: specialty, dtype: int64

En la tabla, providertype esta vacia asi que la terminaremos eliminando. Por otro lago, interventioncategory también presenta campos nullos <br>
y a simple vista es a criterio del medico no depende de la especialidad, la principal caracterísca de la tabla, así que una solución rápida <br>
para esto será rellanar los campos nullos.

Como esta muy repartida la interventioncategory la rellenaremos de manera aleatoria. Otras features como managingphysician y activeupondischarge presentan <br>
poca varianza y finalmente terminaremos eliminandolas y quedandonos con las intervenciones y las specialty.

In [33]:
# Quitar NaN y Uknown de interventioncategory reemplazando los valores por los 3 más frecuentes
def clean_interventioncategory(categ):
    if categ == categ and categ != 'Unknown': # No NaN
        return categ
    else:
        rand = random.randint(0, 2)
        categs = ['I', 'II', 'III']
        return categs[rand]

In [34]:
dfs['carePlanCareProvider'] = dfs['carePlanCareProvider'].drop(columns=[
    'providertype', 
    'managingphysician', 
    'activeupondischarge', 
])

# Rellenamos categoria de Intervención aleatoriamente
dfs['carePlanCareProvider']['interventioncategory']   = dfs['carePlanCareProvider']['interventioncategory'].apply(clean_interventioncategory)

In [35]:
dfs['carePlanCareProvider']

,patientunitstayid,careprovidersaveoffset,specialty,interventioncategory
0,149713,11,family practice,I
1,157016,2,obstetrics/gynecology,I
2,165840,26,internal medicine,I
3,174826,49,critical care medicine (CCM),I
4,174956,3,cardiology,III
...,...,...,...,...
5622,3333069,135,surgery-trauma,II
5623,3333069,3396,critical care medicine (CCM),II
5624,3334564,550,surgery-trauma,II
5625,3334564,-7,surgery-trauma,II


In [36]:
dfs['carePlanCareProvider']['specialty'].value_counts()[:3]

internal medicine    990
hospitalist          549
cardiology           529
Name: specialty, dtype: int64

Nos quedamos con los pacientes que tengan estas especialidades para reducir el número de filas, y lo que haremos será relacionar su <br>
offset en cada caso para saber lo que ha esperado el paciente. A parte, nos quedamos también con el número de intervenciones y el tipo <br>
de intervención para cada uno de los casos.

In [37]:
# Filtramos y nos quedamos con las specialty más comunes
df = dfs['carePlanCareProvider']
dfs['carePlanCareProvider'] = df[df.specialty.isin (['internal medicine', 'cardiology', 'hospitalist'])]

In [38]:
set(dfs["carePlanCareProvider"]["interventioncategory"])

{'I', 'II', 'III', 'IV'}

In [39]:
def has_category_I (row):
    return 1 if row["interventioncategory"] == 'I' else 0
def has_category_II (row):
    return 1 if row["interventioncategory"] == 'II' else 0
def has_category_III (row):
    return 1 if row["interventioncategory"] == 'III' else 0
def has_category_IV (row):
    return 1 if row["interventioncategory"] == 'IV' else 0

def has_hospitalist (row):
    return row['careprovidersaveoffset'] if row['specialty'] == 'hospitalist' else 0

def has_cardiology (row):
    return row['careprovidersaveoffset'] if row['specialty'] == 'cardiology' else 0

def has_internal_medicine (row):
    return row['careprovidersaveoffset'] if row['specialty'] == 'internal medicine' else 0

In [40]:
_dfs = dfs["carePlanCareProvider"]["patientunitstayid"]

# Creamos nuevas columnas
category_I   = dfs["carePlanCareProvider"].apply(lambda row : has_category_I(row), axis=1)
category_II  = dfs["carePlanCareProvider"].apply(lambda row : has_category_II(row), axis=1)
category_III = dfs["carePlanCareProvider"].apply(lambda row : has_category_III(row), axis=1)
category_IV  = dfs["carePlanCareProvider"].apply(lambda row : has_category_IV(row), axis=1)

category_hospital           = dfs["carePlanCareProvider"].apply(lambda row : has_hospitalist(row), axis=1)
category_cardiology         = dfs["carePlanCareProvider"].apply(lambda row : has_cardiology(row), axis=1)
category_internal_medicine  = dfs["carePlanCareProvider"].apply(lambda row : has_internal_medicine(row), axis=1)

# Vemos si hay intervenciones y de que tipo son
_category_I    = pd.DataFrame(category_I, columns=['Intervencion_I'])
_category_II   = pd.DataFrame(category_II, columns=['Intervencion_II'])
_category_III  = pd.DataFrame(category_III, columns=['Intervencion_III'])
_category_IV   = pd.DataFrame(category_IV, columns=['Intervencion_IV'])

# Juntamos intervenciones y agrupamos en base a patientId escogiendo el max (1 or 0)
Intervention = pd.concat([_dfs, _category_I,_category_II,_category_III, _category_IV], axis=1)
Intervention = Intervention.groupby(['patientunitstayid']).max()

# Similar a intervenciones pero con las especialidades más comunes
_category_hospital           = pd.DataFrame(category_hospital, columns=['Categoria_Hospital'])
_category_cardiology         = pd.DataFrame(category_cardiology, columns=['Categoria_Cardiology'])
_category_internal_medicine  = pd.DataFrame(category_internal_medicine, columns=['Categoria_Internal_Medicine'])

# Similar pero sumando 
medicine = pd.concat([_dfs, _category_hospital ,_category_cardiology,_category_internal_medicine ], axis=1)
medicine = medicine.drop_duplicates()
medicine = medicine.groupby(['patientunitstayid']).sum()

dfs['carePlanCareProvider'] = pd.concat([medicine, Intervention], axis=1)
dfs['carePlanCareProvider'] = dfs['carePlanCareProvider'].reset_index()

In [41]:
dfs['carePlanCareProvider']

,patientunitstayid,Categoria_Hospital,Categoria_Cardiology,Categoria_Internal_Medicine,Intervencion_I,Intervencion_II,Intervencion_III,Intervencion_IV
0,141765,14,0,0,1,0,0,0
1,144815,0,0,28,1,0,0,0
2,145427,0,0,153,0,1,0,0
3,148611,0,0,-82,1,0,0,0
4,151179,0,0,2005,0,1,0,0
...,...,...,...,...,...,...,...,...
1120,3351295,0,768,0,0,1,0,0
1121,3351297,0,931,0,0,1,0,0
1122,3352230,0,1996,0,0,1,0,0
1123,3352231,0,534,0,0,1,0,0


Este es el resultado final del pre-procesado, hemos pasado de 5496 a 1125 gracias a la agrupación de las columnas. Además, al igual que en el caso<br>
anterior tenemos una fila para cada paciente.  <br>
Podemos observar que existen valores negativos en los offset, estos representan minutos y por razones obvias no existen minutos  negativos, dado que no son<br>
pocos hemos decidido mantenerlos ya que si ponemos 0 perderíamos mucha información.

---

<center><h2><b>Careplaneol</b></h2></center>

Esta tabla representa los planes relativos al "end of life" de los pacientes.

In [42]:
dfs['carePlanEOL'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15 entries, 0 to 14
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   patientunitstayid       15 non-null     int64
 1   cpleolsaveoffset        15 non-null     int64
 2   cpleoldiscussionoffset  15 non-null     int64
 3   activeupondischarge     15 non-null     bool 
dtypes: bool(1), int64(3)
memory usage: 495.0 bytes


In [43]:
dfs['carePlanEOL']

,patientunitstayid,cpleolsaveoffset,cpleoldiscussionoffset,activeupondischarge
0,1054428,304,0,True
1,1593179,3992,0,True
2,2592641,3998,0,True
3,2611237,303,0,True
4,2621948,987,0,True
5,2630865,3296,0,False
6,2630865,3462,0,True
7,2635556,16603,0,True
8,2635556,14320,0,False
9,2637542,3319,0,True


Esta tabla solo cuenta con 15 filas y no tiene campos nulos asi que no vamos a poder pre-procesar mucho. Más adelante en el procesado  veremos <br>
si es o no es útil pero a simple vista no parece que ayude, simplemente por la poca información que nos aporta (15 filas es muy poco)

---

<center><h2><b>careplangeneral</b></h2></center>

Proposito: Documentación relativa a la planificación de los cuidados, actualizada continuamente durante la estancia de un paciente. <br>

De primeras parece que esta muy relacionada con el objetivo ya que podemos ver que cuidados ha recibido el paciente y el tiempo que ha pasado <br>
hasta recibirlos.

In [44]:
dfs['carePlanGeneral'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33049 entries, 0 to 33147
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   patientunitstayid    33049 non-null  int64 
 1   activeupondischarge  33049 non-null  bool  
 2   cplitemoffset        33049 non-null  int64 
 3   cplgroup             33049 non-null  object
 4   cplitemvalue         32039 non-null  object
dtypes: bool(1), int64(2), object(2)
memory usage: 1.3+ MB


In [45]:
dfs['carePlanGeneral'].head()

,patientunitstayid,activeupondischarge,cplitemoffset,cplgroup,cplitemvalue
0,174826,True,49,Ventilation,Spontaneous - adequate
1,174826,True,49,Care Limitation,Full therapy
2,174826,True,49,Stress Ulcer Prophylaxis,Proton pump inhibitor
3,174826,True,49,Airway,Not intubated/normal airway
4,174826,True,49,DVT Prophylaxis,Compression devices


Para ahorarnos quebraderos de cabeza eliminamos esas 15 filas nulas.

El cplitemvalue no esta muy relacionado con el tipo de cplgroup, es muy cambiante y no va a ayudar al modelo asi que la acabaremos eliminando. <br>
Por otro lado, el activeupondischarge no va a ayudar a ya que buscamos saber a que planes esta sujeto y el tiempo que ha pasado no si esta activo o no <br>
el tratamiento.

In [46]:
# Hay 15 filas con valor nulo en la columna cplitemvalue, asi que eliminamos esas filas
dfs['carePlanGeneral'] = dfs['carePlanGeneral'].dropna()

dfs['carePlanGeneral'] = dfs['carePlanGeneral'].drop(columns=[
    'activeupondischarge', 
    'cplitemvalue' 
], axis=1)

In [47]:
dfs['carePlanGeneral']['cplgroup'].value_counts()[:5]

Ventilation                 3714
DVT Prophylaxis             3296
Airway                      3141
Care Limitation             2961
Stress Ulcer Prophylaxis    2945
Name: cplgroup, dtype: int64

In [48]:
def has_Ventilation (row):
    return row['cplitemoffset'] if row['cplgroup'] == 'Ventilation' else 0

def has_DVT_Prophylaxis (row):
    return row['cplitemoffset'] if row['cplgroup'] == 'DVT Prophylaxis' else 0

def has_Airway (row):
    return row['cplitemoffset'] if row['cplgroup'] == 'Airway' else 0

def has_Care_Limitation (row):
    return row['cplitemoffset'] if row['cplgroup'] == 'Care Limitation' else 0

def has_Stress_Ulcer_Prophylaxis  (row):
    return row['cplitemoffset'] if row['cplgroup'] == 'Stress Ulcer Prophylaxis' else 0

In [49]:
_dfs = dfs["carePlanGeneral"]['patientunitstayid']

category_Ventilation               = dfs["carePlanGeneral"].apply(lambda row : has_Ventilation(row), axis=1)
category_DVT_Prophylaxis           = dfs["carePlanGeneral"].apply(lambda row : has_DVT_Prophylaxis(row), axis=1)
category_Airway                    = dfs["carePlanGeneral"].apply(lambda row : has_Airway(row), axis=1)
category_Care_Limitation           = dfs["carePlanGeneral"].apply(lambda row : has_Care_Limitation(row), axis=1)
category_Stress_Ulcer_Prophylaxis  = dfs["carePlanGeneral"].apply(lambda row : has_Stress_Ulcer_Prophylaxis(row), axis=1)

_category_Ventilation                = pd.DataFrame(category_Ventilation, columns=['Offset_Ventilacion'])
_category_DVT_Prophylaxis            = pd.DataFrame(category_DVT_Prophylaxis, columns=['Offset_DVT_Prophylaxis'])
_category_Airway                     = pd.DataFrame(category_Airway, columns=['Offset_Airway'])
_category_Care_Limitation            = pd.DataFrame(category_Care_Limitation, columns=['Offset_Care_Limitation'])
_category_Stress_Ulcer_Prophylaxis   = pd.DataFrame(category_Stress_Ulcer_Prophylaxis, columns=['Offset_Stress_Ulcer_Prophylaxis'])

categoria = pd.concat([_dfs, _category_Ventilation ,_category_DVT_Prophylaxis,_category_Airway, _category_Care_Limitation, _category_Stress_Ulcer_Prophylaxis ], axis=1)
#categoria = categoria.drop_duplicates()
categoria = categoria.groupby(['patientunitstayid']).sum()
categoria = categoria.reset_index()

dfs['carePlanGeneral'] = categoria
dfs['carePlanGeneral']

,patientunitstayid,Offset_Ventilacion,Offset_DVT_Prophylaxis,Offset_Airway,Offset_Care_Limitation,Offset_Stress_Ulcer_Prophylaxis
0,141764,0,0,0,0,0
1,141765,14,14,14,242,899
2,143870,13,13,13,10,13
3,144815,28,288,28,28,288
4,145427,14,139,14,14,153
...,...,...,...,...,...,...
2450,3351763,3010,52,3010,52,4655
2451,3352230,3640,2225,3640,49,866
2452,3352231,20,277,20,20,20
2453,3352333,4482,10940,4024,9988,10940


Este es el resultado final de la tabla tras el pre-procesado. Tenemos las curas más comunes y el tiempo total que ha pasado para que el paciente <br>
reciviese ese tipo de cura. Volvemos a agrupar los pacientes para tener una fila por paciente, reduciendo el número de filas.

<center><h2><b>careplangoal</b></h2></center>

In [50]:
dfs['carePlanGoal'] = dfs['carePlanGoal'].drop(columns=[
    #'cplgoaloffset', 
    'cplgoalvalue',
    'cplgoalstatus',
    'activeupondischarge'
], axis=1)

dfs['carePlanGoal']

,patientunitstayid,cplgoaloffset,cplgoalcategory
0,1318254,800,Infection/Labs
1,1318254,800,Infection/Labs
2,1318254,800,Infection/Labs
3,1318254,800,Cardiovascular
4,1318254,36,Cardiovascular
...,...,...,...
3628,3158919,1065,Fluid Balance/Treatments
3629,3158919,853,Patient-Family
3630,3158919,1065,Infection/Labs
3631,3158919,853,Fluid Balance/Treatments


In [51]:
dfs['carePlanGoal']['cplgoalcategory'].value_counts()[:6]

Patient-Family              520
Pulmonary                   513
Activity-Safety             433
Cardiovascular              359
Fluid Balance/Treatments    347
Infection/Labs              324
Name: cplgoalcategory, dtype: int64

In [52]:
def has_Patient_Family (row):
    return row['cplgoaloffset'] if row['cplgoalcategory'] == 'Patient-Family' else 0

def has_Pulmonary (row):
    return row['cplgoaloffset'] if row['cplgoalcategory'] == 'Pulmonary' else 0

def has_Fluid_Balance_Treatments (row):
    return row['cplgoaloffset'] if row['cplgoalcategory'] == 'Fluid Balance/Treatments' else 0

def has_Activity_Safety (row):
    return row['cplgoaloffset'] if row['cplgoalcategory'] == 'Activity-Safety' else 0

def has_Cardiovascular  (row):
    return row['cplgoaloffset'] if row['cplgoalcategory'] == 'Cardiovascular' else 0

def has_Infection_Labs  (row):
    return row['cplgoaloffset'] if row['cplgoalcategory'] == 'Infection/Labs' else 0

In [53]:
_dfs = dfs["carePlanGoal"]['patientunitstayid']

category_Patient_Family            = dfs["carePlanGoal"].apply(lambda row : has_Patient_Family(row), axis=1)
category_Pulmonary                 = dfs["carePlanGoal"].apply(lambda row : has_Pulmonary(row), axis=1)
category_Fluid_Balance_Treatments  = dfs["carePlanGoal"].apply(lambda row : has_Fluid_Balance_Treatments(row), axis=1)
category_Activity_Safety           = dfs["carePlanGoal"].apply(lambda row : has_Activity_Safety(row), axis=1)
category_Cardiovascular            = dfs["carePlanGoal"].apply(lambda row : has_Cardiovascular(row), axis=1)
category_Infection_Labs            = dfs["carePlanGoal"].apply(lambda row : has_Infection_Labs(row), axis=1)

_category_Patient_Family           = pd.DataFrame(category_Patient_Family, columns=['Care_Patient_Family'])
_category_Pulmonary                = pd.DataFrame(category_Pulmonary, columns=['Care_Pulmonary'])
category_Fluid_Balance_Treatments  = pd.DataFrame(category_Fluid_Balance_Treatments, columns=['Care_Fluid_Balance_Treatments'])
_category_Activity_Safety          = pd.DataFrame(category_Activity_Safety, columns=['Care_Activity_Safety'])
_category_Cardiovascular           = pd.DataFrame(category_Cardiovascular, columns=['Care_Cardiovascular'])
_category_Infection_Labs           = pd.DataFrame(category_Infection_Labs, columns=['Care_Infection_Labs'])

categoria = pd.concat([_dfs, _category_Patient_Family ,_category_Pulmonary,category_Fluid_Balance_Treatments, _category_Activity_Safety, _category_Cardiovascular, _category_Infection_Labs ], axis=1)
#categoria = categoria.drop_duplicates()
categoria = categoria.groupby(['patientunitstayid']).sum()
categoria = categoria.reset_index()

dfs['carePlanGoal'] = categoria

<center><h2><b>careplaninfectiousdisease</b></h2></center>

In [54]:
dfs['carePlanInfectiousDisease'] = dfs['carePlanInfectiousDisease'].drop(columns=[
    'responsetotherapy', # Solo tiene 2 no nulls
    'treatment', # La mitad son nulls y no es muy útil para empezar
    'activeupondischarge',
    'cplinfectdiseaseoffset'
])

dfs['carePlanInfectiousDisease'] = dfs['carePlanInfectiousDisease'].drop_duplicates()
dfs['carePlanInfectiousDisease'] = dfs['carePlanInfectiousDisease'][dfs['carePlanInfectiousDisease'] != 'Unknown']
dfs['carePlanInfectiousDisease']['infectdiseasesite'] = dfs['carePlanInfectiousDisease']['infectdiseasesite'].fillna('Other')

<center><h2><b>customlab</b></h2></center>

In [55]:
dfs['customLab'] = dfs['customLab'].drop(columns=[
    'labotherresult', # Igual a labothervaluetext
])

# Normalizar a minúsculas
dfs['customLab']['labothervaluetext'] = dfs['customLab']['labothervaluetext'].apply(lambda val: val.lower())

<center><h2><b>diagnosis</b></h2></center>

In [56]:
# Reemplazar NaN por un valor por defecto
dfs['diagnosis']['icd9code'] = dfs['diagnosis']['icd9code'].fillna('_Unknown')

In [57]:
# MANTENER 3 DIAGNOSIS MÁS IMPORTANTES POR SECUENCIA Y PRIORIDAD
new_df = pd.DataFrame(columns=['patientunitstayid', 'diag1', 'diag2', 'diag3', 'additional1', 'additional2', 'additional3'])

def clean_diagnosisstring(group):
    pusID, group_df = group
    priority = { 'Primary': [], 'Major':  [],  'Other': []  }
    offset = { 'Primary': [], 'Major':  [],  'Other': []  }
    
    # Obtener las diagnossis de un paciente
    for i, row in group_df.iterrows():
        if row['diagnosisstring'] == row['diagnosisstring'] and row['diagnosispriority'] == row['diagnosispriority']: # not NaN
            diag = row['diagnosisstring'].split('|')[-1]
            priority_lv = row['diagnosispriority']
            
            priority[priority_lv].append(diag)
            offset[priority_lv].append(row['diagnosisoffset'])
    
    diag_order = priority['Primary'] + priority['Major'] + priority['Other'] + ['None', 'None', 'None', 'None']
    offset_order = offset['Primary'] + offset['Major'] + offset['Other'] + [-1, -1, -1, -1]
    
    return [pusID] + diag_order[:4] + offset_order[:4]

new_df = pd.DataFrame(columns=['patientunitstayid', 'last1', 'last2', 'last3', 'last4', 'last1_off', 'last2_off', 'last3_off', 'last4_off'])
for i,  group in enumerate(dfs['diagnosis'].groupby('patientunitstayid')):
    row = clean_diagnosisstring(group)
    new_df.loc[i] = row

dfs['diagnosis'] = new_df

<center><h2><b>hospital</b></h2></center>

In [58]:
# Reemplazar NaN por un valor por defecto
dfs['hospital']['numbedscategory'] = dfs['hospital']['numbedscategory'].fillna('_Unknown')
dfs['hospital']['region'] = dfs['hospital']['region'].fillna('_Unknown')

<center><h2><b>Infusiondrug</b></h2></center>

In [59]:
# Reemplazar NaN numéricos por un valor por defecto
dfs['infusiondrug']['drugrate'] = dfs['infusiondrug']['drugrate'].fillna(-1)
dfs['infusiondrug']['infusionrate'] = dfs['infusiondrug']['infusionrate'].fillna(-1)
dfs['infusiondrug']['drugamount'] = dfs['infusiondrug']['drugamount'].fillna(-1)
dfs['infusiondrug']['volumeoffluid'] = dfs['infusiondrug']['volumeoffluid'].fillna(-1)
dfs['infusiondrug']['patientweight'] = dfs['infusiondrug']['patientweight'].fillna(-1)

In [60]:
#
# Mantener la última medición para cada paciente
#

def parse_InfussionDrug_info(group):
    pusID, g = group
    systolic, dystolic, mean = -1, -1, -1
    max_offset = -1
    
    for i, row in g.iterrows():
        if row['infusionoffset'] > max_offset:
            max_offset = row['infusionoffset']
    
    return [pusID, max_offset]

new_df = pd.DataFrame(columns=['patientunitstayid', 'lastInfusionDrugOffset'])
for i,  group in enumerate(dfs['infusiondrug'].groupby('patientunitstayid')):
    row = parse_InfussionDrug_info(group)
    new_df.loc[i] = row

dfs['infusiondrug'] = new_df
new_df

,patientunitstayid,lastInfusionDrugOffset
0,243097,215
1,249328,16889
2,250574,847
3,250724,4486
4,257802,575
...,...,...
821,3351763,3990
822,3352230,2307
823,3352231,483
824,3352333,3300


<center><h2><b>intakeoutput</b></h2></center>

In [61]:
dfs['intakeOutput'] = dfs['intakeOutput'].drop(columns=[
    'intaketotal', # Computada en otra variable
    'outputtotal', # Computada en otra variable
    'cellpath', # Redundante en celltext
    'cellvaluetext' # dependiente de cellvaluenumeric
], axis=1)

<center><h2><b>lab</b></h2></center>

In [62]:
# Quitar filas sin laboresult
df = dfs['lab']
dfs['lab'] = df[df['labresult'].notnull()]

dfs['lab'] = dfs['lab'].drop(columns=[
    'labtypeid', # Irrelevante
    'labname', # Irrelevante
    'labresulttext', # Redundante con labResult
    'labresultrevisedoffset' # Reundante con labresultoffset
], axis=1)

<center><h2><b>medication</b></h2></center>

In [63]:
dfs['medication']

,patientunitstayid,drugorderoffset,drugstartoffset,drugivadmixture,drugordercancelled,drugname,drughiclseqno,dosage,routeadmin,frequency,loadingdose,prn,drugstopoffset,gtc
0,141765,134,1396,No,No,WARFARIN SODIUM 5 MG PO TABS,2812.0,5 3,PO,NaN,NaN,No,2739,0
1,141765,1,-188,No,No,5 ML VIAL : DILTIAZEM HCL 25 MG/5ML IV SOLN,182.0,15 3,IV,Once PRN,NaN,Yes,171,38
2,141765,115,856,No,No,ASPIRIN EC 81 MG PO TBEC,1820.0,81 3,PO,Daily,NaN,No,2739,0
3,141765,114,316,No,No,DILTIAZEM HCL 30 MG PO TABS,182.0,30 3,PO,Q6H SCH,NaN,No,2739,0
4,141765,115,856,No,No,LISINOPRIL 5 MG PO TABS,132.0,5 3,PO,Daily,NaN,No,2428,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75599,3353113,3367,3789,No,Yes,NaN,6249.0,1 EACH,TRANSDERM,DAILY,NaN,No,0,97
75600,3353113,2,-55,No,No,NaN,20971.0,100 ML,INTRAVEN,DRIP,NaN,No,2721,83
75601,3353113,2,-2258,No,Yes,LORAZEPAM,4846.0,2-4 MG,INTRAVEN,NaN,NaN,Yes,0,83
75602,3353113,2,-86,No,Yes,NaN,23379.0,20 MG,INTRAMUSC,Q4HRSPRN,NaN,Yes,-65,80


In [64]:
dfs['medication']['drugname'].value_counts()[:10]

ACETAMINOPHEN                                        347
potassium chloride                                   334
POTASSIUM CHLORIDE                                   303
ACETAMINOPHEN 325 MG PO TABS                         294
NaCl 0.9% MBP                                        280
1000 ML FLEX CONT : SODIUM CHLORIDE 0.9 % IV SOLN    270
SODIUM CHLORIDE 0.9%                                 257
FUROSEMIDE                                           247
SODIUM CHLORIDE 0.9 % IV : 1000 ML                   244
TYLENOL                                              235
Name: drugname, dtype: int64

In [65]:
#
# Last time a patient has started a medication (not administrated)
#

patients = {}

for i, row in dfs['medication'].iterrows():
    pusID = row['patientunitstayid']
    orderoff = row['drugstartoffset']
    
    if pusID not in patients:
        patients[pusID] = -1
    
    if orderoff > patients[pusID]:
        patients[pusID] = orderoff

#dfs['medication']['last_med_order'] = 
#dfs['medication'].groupby('patientunitstayid').apply(max)
new_df = pd.DataFrame(columns=['patientunitstayid', 'last_order_offset'])

keys = patients.keys()
new_df['patientunitstayid'] = keys
new_df['last_order_offset'] = list(map(lambda key: patients[key], keys))

dfs['medication'] = new_df

<center><h2><b>microLab</b></h2></center>

In [69]:
dfs['microLab'] = dfs['microLab'].drop(columns=[
    'antibiotic', # TODO: ? Irrelevante
    'sensitivitylevel', # No parece relevante
], axis=1)

<center><h2><b>note</b></h2></center>

In [70]:
# Reemplazar NaN por un valor por defecto
dfs['note']['notetext'] = dfs['note']['notetext'].fillna('_Unknown')

dfs['note'] = dfs['note'].drop(columns=[
    'noteenteredoffset', # Irrelevante
    'notetype', # No parece relevante
    'notevalue', # Parece tener información sobre notetext
], axis=1)

<center><h2><b>nurseAssessment</b></h2></center>

In [71]:
# TODO: ? Parece que hay múltiples assessments de diferentes enfermeras al mismo tiempo con diagnósticos diferentes.
# TODO: Unir celllabel	cellattribute	cellattributevalue
# TODO: Hacer algo con cellattributepath

# Reemplazar NaN por un valor por defecto
dfs['nurseAssessment']['cellattributevalue'] = dfs['nurseAssessment']['cellattributevalue'].fillna('_Unknown')

dfs['nurseAssessment'] = dfs['nurseAssessment'].drop(columns=[
    'nurseassessentryoffset', # Irrelevante
], axis=1)

<center><h2><b>nurseCare</b></h2></center>

In [72]:
dfs['nurseCare'] = dfs['nurseCare'].drop(columns=[
    'nursecareentryoffset', # Irrelevante
    'celllabel' # cellattribute es más detallada
], axis=1)

<center><h2><b>nurseCharting</b></h2></center>

In [73]:
# Hacer algo con esta mierda

dfs['nurseCharting'] = dfs['nurseCharting'].drop(columns=[
    'nursingchartentryoffset' # Irrelevante
], axis=1)

<center><h2><b>pastHistory</b></h2></center>

In [74]:
dfs['pastHistory'] = dfs['pastHistory'].drop(columns=[
    'pasthistoryenteredoffset', # Irrelevante
    'pasthistoryvaluetext' # Igual que pasthistoryvalue
], axis=1)

<center><h2><b>patient</b></h2></center>

In [75]:
#for row in ['gender', 'apacheadmissiondx', 'unitadmitsource', 'unitstaytype' 'unitdischargelocation',]
dfs['patient']['apacheadmissiondx'] = dfs['patient'].apply(lambda row: str(row['apacheadmissiondx']).lower() , axis=1)

In [76]:
dfs['patient']

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,unitadmitsource,unitvisitnumber,unitstaytype,admissionweight,dischargeweight,unitdischargetime24,unitdischargeoffset,unitdischargelocation,unitdischargestatus,uniquepid
0,141764,129391,Female,87,Caucasian,59,91,nan,157.5,23:36:00,...,ICU to SDU,2,stepdown/other,NaN,NaN,18:58:00,344,Home,Alive,002-1039
1,141765,129391,Female,87,Caucasian,59,91,"rhythm disturbance (atrial, supraventricular)",157.5,23:36:00,...,Emergency Department,1,admit,46.5,45.0,13:14:00,2250,Step-Down Unit (SDU),Alive,002-1039
2,143870,131022,Male,76,Caucasian,68,103,"endarterectomy, carotid",167.0,20:46:00,...,Operating Room,1,admit,77.5,79.4,10:00:00,793,Floor,Alive,002-12289
3,144815,131736,Female,34,Caucasian,56,82,"overdose, other toxin, poison or drug",172.7,01:44:00,...,Emergency Department,1,admit,60.3,60.7,20:48:00,1121,Other External,Alive,002-1116
4,145427,132209,Male,61,Caucasian,68,103,"gi perforation/rupture, surgery for",177.8,23:48:00,...,Operating Room,1,admit,91.7,93.1,22:47:00,1369,Floor,Alive,002-12243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2515,3351763,2741766,Female,62,Caucasian,459,1108,"fistula/abscess, surgery for (not inflammatory...",165.1,16:08:00,...,Operating Room,1,admit,134.5,133.3,19:24:00,5394,Step-Down Unit (SDU),Alive,035-10391
2516,3352230,2742186,Male,41,African American,458,1107,"cabg alone, coronary artery bypass grafting",177.8,21:21:00,...,Operating Room,2,transfer,127.0,128.5,21:34:00,4261,Telemetry,Alive,035-10089
2517,3352231,2742186,Male,41,African American,458,1104,"infarction, acute myocardial (mi)",177.8,21:21:00,...,Direct Admit,1,admit,127.0,135.2,22:26:00,1369,Other ICU,Alive,035-10089
2518,3352333,2742269,Male,72,Caucasian,458,1111,gi obstruction,177.8,20:00:00,...,Other Hospital,1,admit,68.3,66.5,17:26:00,4166,Floor,Alive,035-10041


In [77]:
#
# Añadir a paciente la media del n de minutos que ha estado en la UCI
#
def parse_avg_unit_stay(row, df):
    total_mins = 0
    total_entries = 0
    
    previous_visits_df = df[
        df['patienthealthsystemstayid'] == row['patienthealthsystemstayid']
    ][
        df['patientunitstayid'] < row['patientunitstayid']
    ]
    
    # Obtener las diagnossis de un paciente
    for i, row in previous_visits_df.iterrows():
        mins = row['unitdischargeoffset']
        total_mins += mins
        total_entries += 1
    
    return 0 if total_mins == 0 else total_mins//total_entries # Integer division

avg_unit_stay = []
for i,  row in dfs['patient'].iterrows():
    n = parse_avg_unit_stay(row, dfs['patient'])
    avg_unit_stay.append(n)

dfs['patient']['avg_unit_stay'] = avg_unit_stay

/tmp/ipykernel_27043/2954303803.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  previous_visits_df = df[


In [78]:
#
# Añadir a paciente la media del n de minutos que ha estado en el hospital
#
def parse_avg_hospital_stay(row, df):
    total_mins = 0
    total_entries = 0
    
    previous_visits_df = df[
        df['patienthealthsystemstayid'] == row['patienthealthsystemstayid']
    ][
        df['patientunitstayid'] < row['patientunitstayid']
    ]
    
    # Obtener las diagnossis de un paciente
    for i, row in previous_visits_df.iterrows():
        mins = row['hospitaldischargeoffset']
        total_mins += mins
        total_entries += 1
    
    return 0 if total_mins == 0 else total_mins//total_entries # Integer division

avg_unit_stay = []
for i,  row in dfs['patient'].iterrows():
    n = parse_avg_hospital_stay(row, dfs['patient'])
    avg_unit_stay.append(n)

dfs['patient']['avg_hospital_stay'] = avg_unit_stay

/tmp/ipykernel_27043/4073688880.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  previous_visits_df = df[


In [79]:
#
# Añadir BMI en admisión
#
def bmi(row):
    height = row['admissionheight']
    weight = row['admissionweight']
    
    if height != height or weight != weight: # has NaN
        return -1
    else:
        return height/weight

dfs['patient']['admission_bmi'] = dfs['patient'].apply(bmi, axis=1)

In [80]:
import math

def clean_age(row):
    age = row['age']
    
    if age == '':
        return 50 # TODO: Median
    elif age == '> 89':
        return 100
    elif math.isnan(float(age)):
        return 50 # ""
    else:
        return int(age)

In [81]:
#Sepsis, pulmonary              -> 121                                                                                                                                                                    121
#Diabetic ketoacidosis          -> 105                                                                                                                                                                 105
#CHF, congestive heart failure  -> 102   

def randomApache():
    num = random.randint(1, 3)
    
    if(num == 1): 
        return 'CHF, congestive heart failure'
    
    elif (num == 2): 
        return 'Diabetic ketoacidosis'
    
    else: return 'Sepsis, pulmonary'
     
def clean_apacheadmissiondx(row):
    apache = row['apacheadmissiondx']
    
    if apache != apache:
        return randomApache() # ""
    else:
        return apache

In [82]:
# 177.80    151
# 172.70    146
# 167.60    133
# 165.10    127
                                                                                                                                                    
def randomAdmissionheight():
    num = random.randint(1, 4)
    
    if(num == 1): 
        return 177.80
    
    elif (num == 2): 
        return 172.70
    
    elif (num == 3): 
        return 167.60
    
    else: return 165.10
     
def clean_admissionheight(row):
    admissionheight = row['admissionheight']
    
    if math.isnan(float(admissionheight)):
        return randomAdmissionheight() # ""
    else:
        return admissionheight

In [83]:
# Random
def clean_gender(row):
    gender = row['gender']
    
    if gender != gender:
        if random.randint(1, 2) % 2 == 0: return 'Male'
        else: return 'Famale'
    else:
        return gender

In [84]:
# Caucasian
def clean_ethnicity(row):
    ethnicity = row['ethnicity']
    
    if ethnicity != ethnicity:
        return 'Caucasian'
    else:
        return ethnicity

In [85]:
# Emergency Department
def clean_hospitaladmitsource(row):
    hospitalAdmit = row['hospitaladmitsource']
    
    if hospitalAdmit != hospitalAdmit:
        return 'Emergency Department' # ""
    else:
        return hospitalAdmit


In [86]:
def clean_admissionweight(row, df):
    admissionweight = row['admissionweight']
    
    if math.isnan(float(admissionweight)):
        return  df[df['age']==row['age']]['admissionweight'].median()
    else:
        return admissionweight    

In [87]:
# TODO: Feature idea - BMI
# TODO: ? Cual es más importante, unitDischargeOffset u hospitalDischargeOffset  

# Drop
dfs['patient'] = dfs['patient'].drop(columns=[
    'wardid', # ? Tiene pinta de que no va a ser muy importante
    'patienthealthsystemstayid', # Irrelevante
    'hospitaldischargeyear', # Irrelevante. Solo hay datos de los años 2014 y 2015
    'hospitaladmittime24', # Drop a priori. No parece importante
    'hospitaldischargetime24', # Drop a priori. No parece importante
    'hospitaldischargelocation', # No parece muy relevante
    'unittype', # Parece que ya está presente en otras partes
    'unitadmittime24', # No parece muy relevante
    'unitadmitsource', # ?
    'unitstaytype', # No parece muy relevante
    'dischargeweight', # No parece que sea muy relevante el peso cuando esté muerto
    'unitdischargetime24', # Irrelevante
    'unitdischargelocation', # Irrelevante
    'hospitaldischargestatus', # Irrelevante
    'unitdischargestatus',
    'uniquepid'
], axis=1)

# Eliminar NaN TODO
#dfs['patient'] = dfs['patient'].dropna()

# Age
dfs['patient']['age']                 = dfs['patient'].apply(clean_age, axis=1)

# Apacheadmissiondx
dfs['patient']['apacheadmissiondx']   = dfs['patient'].apply(clean_apacheadmissiondx, axis=1)

# Hospitaladmitsource
dfs['patient']['hospitaladmitsource'] = dfs['patient'].apply(clean_hospitaladmitsource, axis=1)

# Gender
dfs['patient']['gender']              = dfs['patient'].apply(clean_gender, axis=1)

# ethnicity
dfs['patient']['ethnicity']           = dfs['patient'].apply(clean_ethnicity, axis=1)

# admissionheight
dfs['patient']['admissionheight']     = dfs['patient'].apply(clean_admissionheight, axis=1)

# admissionweight -> OUTLAYERS TODO:
dfs['patient']['admissionweight']     = dfs['patient'].apply(lambda row : clean_admissionweight(row, dfs['patient']), axis=1)

<center><h2><b>physicalExam</b></h2></center>

In [88]:
#set(dfs['physicalExam']['physicalexamvalue'])
dfs['physicalExam']['physicalexamtext'] = dfs['physicalExam']['physicalexamtext'].fillna('_Unknown')
# BPDiastolic current
# BPDiastolic highest
# BPDiastolic lowest
# BPSystolic current
# BPSystolic highest
# BPSystolic lowest

# Blood loss
# Dialysis net

# O2 Sat current
# O2 Sat lowest
# O2 Sat highest
# Urine
# Intubated
# Comatose
# Ventilated

def physical_exam_info(group):
    pusID, group_df = group
    d = {
        'BP (diastolic) Current': -1,
        'BP (diastolic) Highest': -1,
        'BP (diastolic) Lowest' : -1,
        
        'BP (systolic) Current': -1,
        'BP (systolic) Highest': -1,
        'BP (systolic) Lowest' : -1,
        
        'Blood Loss': -1,
        'Dialysis Net': -1,
        'O2 Sat% Current': -1,
        'O2 Sat% Highest': -1,
        'O2 Sat% Lowest': -1,
        'Urine': -1,
        'Intubated': -1,
        'Comatose': -1,
        'Ventilated': -1,
        
        'Motor Score': -1,
        'Verbal Score': -1,
        'Eyes Score': -1
    }
    
    last_offset = float('inf')
    
    # Obtener las diagnossis de un paciente
    for i, row in group_df.iterrows():
        off = row['physicalexamoffset']
        
        if off < last_offset:
            last_offset = off
            diag = row['physicalexamvalue']
            val = row['physicalexamtext']
            
            
            if diag in d:
                d[diag] = val
    
    for i, row in group_df.iterrows():
        path = row['physicalexampath']
        part = path.split('/')[-2]
            
        if part in d:
            d[part] = row['physicalexamtext']
    
    return [pusID, d['BP (diastolic) Current'], d['BP (diastolic) Highest'], d['BP (diastolic) Lowest'],
            d['BP (systolic) Current'], d['BP (systolic) Highest'], d['BP (systolic) Lowest'], d['Blood Loss'],
            d['Dialysis Net'], d['O2 Sat% Current'], d['O2 Sat% Highest'], d['O2 Sat% Lowest'], d['Urine'], d['Intubated'], d['Comatose'], d['Ventilated'],
            d['Motor Score'], d['Verbal Score'], d['Eyes Score']]

new_df = pd.DataFrame(columns=[
    'patientunitstayid', 'BPD_Current', 'BPD_Highest', 'BPD_Lowest', 'BPS_Current', 'BPS_Highest', 'BPS_Lowest', 'Blood_Loss', 'Dialysis_Net',
    'O2Sat_Current', 'O2Sat_Highest', 'O2Sat_Lowest', 'Urine', 'Intubated', 'Comatose', 'Ventilated', 'Motor', 'Verbal', 'Eyes'
])

for i,  group in enumerate(dfs['physicalExam'].groupby('patientunitstayid')):
    row = physical_exam_info(group)
    new_df.loc[i] = row

dfs['physicalExam'] = new_df

<center><h2><b>respiratoryCare</b></h2></center>

In [89]:
# Me quedo con el numero máximo de cada paciente
def clean_respiratoryCare(row, df):
    respiratoryCare = row['currenthistoryseqnum']
           
    return  df[df['patientunitstayid']==row['patientunitstayid']]['currenthistoryseqnum'].max()

# Clasifico
def range_currenthistoryseqnum(row, df):
    median = 3.00  # df['currenthistoryseqnum'].median()
    currenthistoryseqnum = row ['currenthistoryseqnum']
    
    if (currenthistoryseqnum<median):
        return 'Low'
    elif currenthistoryseqnum == median:
        return 'Mid'
    else:
        return 'High' 

In [90]:
dfs['respiratoryCare']['currenthistoryseqnum'] = dfs['respiratoryCare'].apply(lambda row : clean_respiratoryCare(row, dfs['respiratoryCare']), axis=1)
dfs['respiratoryCare']['currenthistoryseqnum'] = dfs['respiratoryCare'].apply(lambda row : range_currenthistoryseqnum(row, dfs['respiratoryCare']), axis=1)

dfs['respiratoryCare'] = dfs['respiratoryCare'].drop(columns=[
    'airwaysize', 'airwayposition', 'cuffpressure', 'apneaparams', 'lowexhmvlimit', 'hiexhmvlimit', 'lowexhtvlimit', 'hipeakpreslimit', 'lowpeakpreslimit',
    'hirespratelimit', 'lowrespratelimit', 'sighpreslimit', 'lowironoxlimit', 'highironoxlimit', 'meanairwaypreslimit', 'peeplimit', 'cpaplimit',
    'setapneainterval', 'setapneatv', 'setapneaippeephigh', 'setapnearr', 'setapneapeakflow', 'setapneainsptime', 'setapneaie', 'setapneafio2', 
    'respcarestatusoffset', 'airwaytype', 'ventstartoffset', 'ventstartoffset', 'ventstartoffset', 'priorventendoffset', 'ventendoffset', 'priorventstartoffset'
], axis=1)

#
# TODO: airwaytype tiene nulls
#

<center><h2><b>respiratoryCharting</b></h2></center>

In [91]:
# TODO: ?

dfs['respiratoryCharting'] = dfs['respiratoryCharting'].drop(columns=[
    'respchartentryoffset' # Irrelevante
], axis=1)

<center><h2><b>treatment</b></h2></center>

In [92]:
# TODO: ?

dfs['treatment'] = dfs['treatment'].drop(columns=[
    'activeupondischarge', # Irrelevante    
], axis=1)

<center><h2><b>vitalAperiodic</b></h2></center>

In [93]:
#
# Mantener la última medición para cada paciente
#

def parse_vap_info(group):
    pusID, g = group
    systolic, dystolic, mean = -1, -1, -1
    max_offset = -1
    
    for i, row in g.iterrows():
        if row['observationoffset'] > max_offset:
            max_offset = row['observationoffset']
            systolic = row['noninvasivesystolic']
            dystolic = row['noninvasivediastolic']
            mean = row['noninvasivemean']
    
    return [pusID, max_offset, systolic, dystolic, mean]

new_df = pd.DataFrame(columns=['patientunitstayid', 'last_aperiodic_off', 'last_aperiodic_systolic', 'last_aperiodic_diastolic', 'last_aperiodic_mean'])
for i,  group in enumerate(dfs['vitalAperiodic'].groupby('patientunitstayid')):
    row = parse_vap_info(group)
    new_df.loc[i] = row

dfs['vitalAperiodic'] = new_df
new_df

,patientunitstayid,last_aperiodic_off,last_aperiodic_systolic,last_aperiodic_diastolic,last_aperiodic_mean
0,141764.0,334.0,153.0,78.0,103.0
1,141765.0,2204.0,175.0,77.0,102.0
2,143870.0,754.0,107.0,54.0,78.0
3,144815.0,1043.0,124.0,70.0,93.0
4,145427.0,1180.0,132.0,71.0,95.0
...,...,...,...,...,...
2326,3351763.0,5310.0,139.0,67.0,96.0
2327,3352230.0,4227.0,144.0,67.0,94.0
2328,3352231.0,804.0,146.0,78.0,101.0
2329,3352333.0,4097.0,90.0,50.0,64.0


<center><h2><b>vitalPeriodic</b></h2></center>

In [94]:
#
# Mantener la última medición para cada paciente
#

def parse_vp_info(group):
    pusID, g = group
    max_offsets = { 'temperature': 0, 'sao2': 0, 'respiration': 0, 'cvp': 0, 'heartrate': 0 }
    measures = { 'temperature': -1, 'sao2': -1, 'respiration': -1, 'cvp':-1, 'heartrate': -1 }
    
    for i, row in g.iterrows():
        off = row['observationoffset']
        
        for measure in ['temperature', 'sao2', 'respiration', 'cvp', 'heartrate']:
            if off < max_offsets[measure] and row[measure] == row[measure]:
                max_offsets[measure] = off
                measures[measure] = row[measure]
    
    return [pusID, measures['temperature'], measures['sao2'], measures['respiration'], measures['cvp'], measures['heartrate']] 

new_df = pd.DataFrame(columns=['patientunitstayid', 'temperature', 'sao2', 'respiration', 'cvp', 'heartrate'])
for i,  group in enumerate(dfs['vitalPeriodic'].groupby('patientunitstayid')):
    row = parse_vp_info(group)
    new_df.loc[i] = row

dfs['vitalPeriodic'] = new_df

---

<center><h2><b>Eliminar duplicados</b></h2></center>

In [96]:
for df_name in dfs:
    n = len(dfs[df_name].duplicated())
    dfs[df_name] = dfs[df_name].drop_duplicates()
    
    diff = n - len(dfs[df_name].duplicated())
    if diff > 0:
        print(df_name, ':', diff)


lab : 3989
microLab : 122
note : 27
pastHistory : 4
respiratoryCare : 5041
treatment : 137


---

<center><h2><b>Exportar a CSV</b></h2></center>

In [97]:
for df_name in dfs:
    dfs[df_name].to_csv('../db/csv_clean/' + df_name + '.csv', index=False)

---

<center><h2><b>Exportar a SQL</b></h2></center>

In [98]:
import os

sqlite_path = "../db/sqlite/eicu_v2_0_1_clean.sqlite3"
os.remove(sqlite_path)
conn = db.connect(sqlite_path)

for df_name in dfs:
    dfs[df_name].to_sql(df_name, conn, index=False)